Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploying a web service to Azure Kubernetes Service (AKS)
In this notebook, we show the following steps for deploying a web service using AzureML:
- Provision an AKS cluster (one time action)
- Deploy the service
- Test the web service
- Scale up the service

In [1]:
from azure_utils.machine_learning.realtime.kubernetes import get_or_create_service

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


AML will use the following information to create an image, provision a cluster and deploy a service. Replace the 
values in the following cell with your information.

In [14]:
??get_or_create_service

In [ ]:
aks_service = get_or_create_aks_service()


In [4]:
from azureml.core.authentication import MsiAuthentication
from azureml.core import Workspace

msi_auth = MsiAuthentication()
ws = Workspace(subscription_id="109e56d8-d599-4905-ab89-be3f6c7e1662",
               resource_group="trial2",
               workspace_name="experiment2ml",
               auth=msi_auth)

In [7]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
#prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)

aks_name = 'myaks2'
# Create the cluster
# aks_target = ComputeTarget.create(workspace = ws,
#                                     name = aks_name,
#                                     provisioning_configuration = prov_config)
aks_target = ComputeTarget(workspace = ws,name = aks_name)
# Wait for the create process to complete
#aks_target.wait_for_completion(show_output = True)

In [2]:
??get_or_create_service

In [3]:
from azureml.core.authentication import MsiAuthentication
from azureml.core import Workspace
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.image import Image
msi_auth = MsiAuthentication()
ws = Workspace(subscription_id="109e56d8-d599-4905-ab89-be3f6c7e1662",
               resource_group="trial2",
               workspace_name="experiment2ml",
               auth=msi_auth)


In [10]:
from azureml.core.compute import AksCompute, ComputeTarget
aks_target = AksCompute(ws,"myaks2")

In [14]:
aks_config = AksWebservice.deploy_configuration(num_replicas=2, cpu_cores=1)
image = ws.images["imageml5"]
aks_service = Webservice.deploy_from_image(workspace=ws, name="aksservice", image=image,deployment_config=aks_config, deployment_target=aks_target)
aks_service.wait_for_deployment(show_output=True)

/anaconda/envs/az-ml-realtime-score/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'show_output' is not defined

In [15]:
aks_service.wait_for_deployment(show_output=True)

Running..................
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [ ]:
def get_or_create_service(configuration_file: str = project_configuration_file, aks_target=None, num_replicas: int = 2,
                          cpu_cores: int = 1, show_output: bool = True) -> AksWebservice:
    """
    Get or Create new Machine Learning Webservice

    :param configuration_file: Project configuration settings file. default: project.yml
    :param aks_target: Kubernetes compute to target for deployment
    :param num_replicas: number of replicas in Kubernetes cluster. default: 2
    :param cpu_cores: cpu cores for web service. default: 1
    :param show_output: toggle on/off standard output. default: `True`
    :return: New or Existing Kubernetes Web Service
    """
    project_configuration = ProjectConfiguration(configuration_file)
    assert project_configuration.has_settings("aks_service_name")
    assert project_configuration.has_settings("image_name")

    aks_service_name = project_configuration.get_value("aks_service_name")
    image_name = project_configuration.get_value("image_name")

    workspace = get_or_create_workspace_from_project(project_configuration, show_output=show_output)

    if aks_service_name in workspace.webservices:
        return workspace.webservices[aks_service_name]

    aks_config = AksWebservice.deploy_configuration(num_replicas=num_replicas, cpu_cores=cpu_cores)

    assert workspace.images[image_name]
    image = workspace.images[image_name]

    deploy_from_image_start = time.time()
    aks_service = Webservice.deploy_from_image(workspace=workspace, name=aks_service_name, image=image,
                                               deployment_config=aks_config, deployment_target=aks_target)
    aks_service.wait_for_deployment(show_output=show_output)
    if show_output:
        deployment_time_secs = str(time.time() - deploy_from_image_start)
        print("Deployed Image with name "
              + aks_service_name + ". Took " + deployment_time_secs + " seconds.")
        print(aks_service.state)
        print(aks_service.get_logs())
    return aks_service

Next, we will test the [throughput of the web service](06_SpeedTestWebApp.ipynb).